In [ ]:
pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf


In [ ]:
df = sns.load_dataset('iris')
df.head()
x = df.drop(columns=['sepal_length'])
y = df['sepal_length']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
le = LabelEncoder()
x_train['species'] = le.fit_transform(x_train['species'])
x_test['species']= le.transform(x_test['species'])

In [ ]:
# Define the model-building function
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1],)))  # Input layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer (regression)
    model.compile(optimizer=SGD(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mae',
                  metrics=['mae'])
    return model

# Define the tuner
tuner = kt.RandomSearch(
    build_model,  # Pass the model-building function
    objective='val_mae',
    max_trials=10,
    executions_per_trial=1,
    directory='test',
    project_name='iris_reg'
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Get the best hyperparameters and build the best model
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)

# Train the best model
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the best model
loss, mae = best_model.evaluate(x_test, y_test)
print(f"Test MAE: {mae}")

Trial 10 Complete [00h 00m 03s]
val_mae: 0.3443584740161896

Best val_mae So Far: 0.3165915906429291
Total elapsed time: 00h 00m 28s
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 6.1992 - mae: 6.1992 - val_loss: 4.6547 - val_mae: 4.6547
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 4.0446 - mae: 4.0446 - val_loss: 2.2820 - val_mae: 2.2820
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.6522 - mae: 1.6522 - val_loss: 0.7737 - val_mae: 0.7737
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.8149 - mae: 0.8149 - val_loss: 0.6909 - val_mae: 0.6909
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.7196 - mae: 0.7196 - val_loss: 0.6090 - val_mae: 0.6090
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.5999 - mae: 0.5999 - val_loss: 0.5506 - val_mae: 0.5506
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.5009 - mae: 0.5009 - val_loss: 0.4574 - val_mae: 0.4574
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.463